## Environment

In [4]:
import os
import time

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from tensorflow import keras

## Prepare Data

In [5]:
housing = fetch_california_housing()
housing

In [14]:
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)
del X_train_full, y_train_full

In [15]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

## Sequantial Model

In [38]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation='relu', input_shape=X_train.shape[1:]),
    keras.layers.Dense(1)
])

In [39]:
model.compile(loss='mse', optimizer='sgd')

In [40]:
history = model.fit(X_train, y_train, epochs=20,
                   validation_data=(X_valid, y_valid))

Epoch 1/20
363/363 [==============================] - 0s 1ms/step - loss: 0.7940 - val_loss: 0.4988
Epoch 2/20
363/363 [==============================] - 0s 980us/step - loss: 0.4899 - val_loss: 0.4481
Epoch 3/20
363/363 [==============================] - 0s 950us/step - loss: 0.4556 - val_loss: 0.4417
Epoch 4/20
363/363 [==============================] - 0s 983us/step - loss: 0.4436 - val_loss: 0.4545
Epoch 5/20
363/363 [==============================] - 0s 968us/step - loss: 0.4341 - val_loss: 0.4177
Epoch 6/20
363/363 [==============================] - 0s 953us/step - loss: 0.4236 - val_loss: 0.4103
Epoch 7/20
363/363 [==============================] - 0s 977us/step - loss: 0.4164 - val_loss: 0.4113
Epoch 8/20
363/363 [==============================] - 0s 952us/step - loss: 0.4105 - val_loss: 0.4009
Epoch 9/20
363/363 [==============================] - 0s 947us/step - loss: 0.4069 - val_loss: 0.3967
Epoch 10/20
363/363 [==============================] - 0s 1ms/step - loss: 0.4682 - 

In [41]:
mse_test_sequential = model.evaluate(X_test, y_test)
mse_test_sequential

162/162 [==============================] - 0s 703us/step - loss: 0.3910


0.3639618754386902

## Wide&Deep Model

In [36]:
input_ = keras.layers.Input(X_train.shape[1])
hidden1 = keras.layers.Dense(30, activation='relu')(input_)
hidden2 = keras.layers.Dense(30, activation='relu')(hidden1)
concat = keras.layers.Concatenate()([input_, hidden2])
output = keras.layers.Dense(1)(concat)
model = keras.Model(inputs=[input_], outputs=[output])

In [42]:
model.compile(loss='mse', optimizer='sgd')

In [43]:
model.fit(X_train, y_train, 
          epochs=20, validation_data=(X_valid, y_valid))

Epoch 1/20
363/363 [==============================] - 0s 1ms/step - loss: 0.3733 - val_loss: 0.3892
Epoch 2/20
363/363 [==============================] - 0s 958us/step - loss: 0.3730 - val_loss: 0.3691
Epoch 3/20
363/363 [==============================] - 0s 988us/step - loss: 0.3708 - val_loss: 0.3719
Epoch 4/20
363/363 [==============================] - 0s 953us/step - loss: 0.3749 - val_loss: 0.3648
Epoch 5/20
363/363 [==============================] - 0s 953us/step - loss: 0.3648 - val_loss: 0.3638
Epoch 6/20
363/363 [==============================] - 0s 980us/step - loss: 0.3667 - val_loss: 0.3637
Epoch 7/20
363/363 [==============================] - 0s 961us/step - loss: 0.3639 - val_loss: 0.4380
Epoch 8/20
363/363 [==============================] - 0s 961us/step - loss: 0.3638 - val_loss: 0.3666
Epoch 9/20
363/363 [==============================] - 0s 986us/step - loss: 0.3622 - val_loss: 0.3570
Epoch 10/20
363/363 [==============================] - 0s 973us/step - loss: 0.3595 

In [44]:
mse_test_deepwide = model.evaluate(X_test, y_test)
mse_test_deepwide

162/162 [==============================] - 0s 814us/step - loss: 0.3579


0.3578910827636719

## Direct Feature Input Model 

In [57]:
input_A = keras.layers.Input(shape=[5], name='direct_input')
input_B = keras.layers.Input(shape=[6], name='deep_input')
hidden1 = keras.layers.Dense(units=30, activation='relu')(input_B)
hidden2 = keras.layers.Dense(units=30, activation='relu')(hidden1)
concat = keras.layers.Concatenate(name='concatenation')([input_A, hidden2])
output = keras.layers.Dense(1, name='output')(concat)
model = keras.Model(inputs=[input_A, input_B], outputs=[output])
model.summary()

Model: "functional_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
deep_input (InputLayer)         [(None, 6)]          0                                            
__________________________________________________________________________________________________
dense_27 (Dense)                (None, 30)           210         deep_input[0][0]                 
__________________________________________________________________________________________________
direct_input (InputLayer)       [(None, 5)]          0                                            
__________________________________________________________________________________________________
dense_28 (Dense)                (None, 30)           930         dense_27[0][0]                   
_______________________________________________________________________________________

In [59]:
X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]

In [58]:
model.compile(loss='mean_squared_error', optimizer='sgd')

In [61]:
history = model.fit((X_train_A, X_train_B), y_train,
                    epochs=20,
                    validation_data=((X_valid_A, X_valid_B), y_valid))

Epoch 1/20
363/363 [==============================] - 0s 1ms/step - loss: 0.8213 - val_loss: 0.5466
Epoch 2/20
363/363 [==============================] - 0s 1ms/step - loss: 0.5399 - val_loss: 0.5288
Epoch 3/20
363/363 [==============================] - 0s 1ms/step - loss: 0.5052 - val_loss: 0.4859
Epoch 4/20
363/363 [==============================] - 0s 1ms/step - loss: 0.4992 - val_loss: 0.4659
Epoch 5/20
363/363 [==============================] - 0s 1ms/step - loss: 0.4842 - val_loss: 0.4495
Epoch 6/20
363/363 [==============================] - 1s 1ms/step - loss: 0.4609 - val_loss: 0.4629
Epoch 7/20
363/363 [==============================] - 0s 1ms/step - loss: 0.4562 - val_loss: 0.5200
Epoch 8/20
363/363 [==============================] - 0s 1ms/step - loss: 0.4446 - val_loss: 0.4396
Epoch 9/20
363/363 [==============================] - 0s 1ms/step - loss: 0.4369 - val_loss: 0.4257
Epoch 10/20
363/363 [==============================] - 0s 1ms/step - loss: 0.4315 - val_loss: 0.4098

In [62]:
mse_model_direct_input = model.evaluate((X_test_A, X_test_B), y_test)
mse_model_direct_input

162/162 [==============================] - 0s 777us/step - loss: 0.3958


0.3957987129688263

## Model with multiple outputs

In [93]:
input_A = keras.layers.Input(shape=[5], name='DirectInput')
input_B = keras.layers.Input(shape=[6], name='DeepInput')
hidden1 = keras.layers.Dense(units=30, activation='relu', name='Dense1')(input_B)
hidden2 = keras.layers.Dense(units=30, activation='relu', name='Dense2')(hidden1)
concat = keras.layers.Concatenate()([input_A, hidden2])
output = keras.layers.Dense(1, name='output')(concat)
output_aux = keras.layers.Dense(1, name='output_other')(hidden2)
model = keras.Model(inputs=[input_A, input_B], outputs=[output, output_aux])
model.summary()

Model: "functional_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
DeepInput (InputLayer)          [(None, 6)]          0                                            
__________________________________________________________________________________________________
Dense1 (Dense)                  (None, 30)           210         DeepInput[0][0]                  
__________________________________________________________________________________________________
DirectInput (InputLayer)        [(None, 5)]          0                                            
__________________________________________________________________________________________________
Dense2 (Dense)                  (None, 30)           930         Dense1[0][0]                     
______________________________________________________________________________________

In [94]:
X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]

In [95]:
model.compile(loss=['mse', 'mse'], optimizer='sgd')

In [96]:
model.fit((X_train_A, X_train_B), (y_train, y_train),
         epochs=20,
         validation_data=((X_valid_A, X_valid_B), (y_valid, y_valid)))

Epoch 1/20
363/363 [==============================] - 1s 2ms/step - loss: 2.3227 - output_loss: 1.0454 - output_other_loss: 1.2774 - val_loss: 1.2085 - val_output_loss: 0.5325 - val_output_other_loss: 0.6760
Epoch 2/20
363/363 [==============================] - 0s 1ms/step - loss: 1.0860 - output_loss: 0.4720 - output_other_loss: 0.6141 - val_loss: 1.0182 - val_output_loss: 0.4408 - val_output_other_loss: 0.5774
Epoch 3/20
363/363 [==============================] - 0s 1ms/step - loss: 1.1038 - output_loss: 0.4747 - output_other_loss: 0.6290 - val_loss: 0.9598 - val_output_loss: 0.4243 - val_output_other_loss: 0.5355
Epoch 4/20
363/363 [==============================] - 0s 1ms/step - loss: 0.9665 - output_loss: 0.4350 - output_other_loss: 0.5316 - val_loss: 0.8953 - val_output_loss: 0.4102 - val_output_other_loss: 0.4851
Epoch 5/20
363/363 [==============================] - 1s 2ms/step - loss: 1.0555 - output_loss: 0.5289 - output_other_loss: 0.5266 - val_loss: 0.8603 - val_output_loss:

In [79]:
mse_model_two_outputs = model.evaluate((X_test_A, X_test_B), (y_test, y_test))
mse_model_two_outputs[1:]

162/162 [==============================] - 0s 869us/step - loss: 0.8394 - output_loss: 0.3873 - output_other_loss: 0.4522


[0.3872627317905426, 0.4521538317203522]

## Wide&Deep Model sub-class implementation

In [90]:
class WideAndDeepModel(keras.Model):
    def __init__(self, units=30, activation='relu', **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(units, activation=activation)
        self.hidden2 = keras.layers.Dense(units, activation=activation)
        self.main_output = keras.layers.Dense(1)
        self.aux_output = keras.layers.Dense(1)
        
    def call(self, inputs):
        input_A, input_B = inputs
        hidden1 = self.hidden1(input_B)
        hidden2 = self.hidden2(hidden1)
        concat = keras.layers.Concatenate()([input_A, hidden2])
        main_output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        return main_output, aux_output

In [91]:
model = WideAndDeepModel()

In [92]:
model.call(inputs=(X_train_A, X_train_B))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



(<tf.Tensor: shape=(11610, 1), dtype=float32, numpy=
 array([[-0.06662218],
        [-0.6289364 ],
        [ 0.23321314],
        ...,
        [ 0.3478999 ],
        [-0.45993525],
        [-0.66158205]], dtype=float32)>,
 <tf.Tensor: shape=(11610, 1), dtype=float32, numpy=
 array([[-0.44738716],
        [-0.04426318],
        [-0.20813489],
        ...,
        [-0.24637066],
        [-1.2469165 ],
        [-0.26630157]], dtype=float32)>)

## Saving Model

In [99]:
model_name = 'california_housing_model.h5'

In [100]:
model.save(model_name)

In [101]:
model = keras.models.load_model(model_name)

In [104]:
!del california_housing_model.h5

## Saving checkpoints during model training

In [107]:
model = keras.models.Sequential([
    keras.layers.Dense(30, 'relu'),
    keras.layers.Dense(30, 'relu'),
    keras.layers.Dense(1)
])
model.compile(loss='mse', optimizer='sgd')

In [112]:
checkpoint_cb = keras.callbacks.ModelCheckpoint(model_name)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10,
                                                  restore_best_weights=True)

In [114]:
history = model.fit(X_train, y_train,
                    validation_data=(X_valid, y_valid),
                    epochs=100,
                    callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/100
363/363 [==============================] - 0s 1ms/step - loss: 0.2996 - val_loss: 0.3022
Epoch 2/100
363/363 [==============================] - 0s 1ms/step - loss: 0.3018 - val_loss: 0.3059
Epoch 3/100
363/363 [==============================] - 0s 1ms/step - loss: 0.2981 - val_loss: 0.3001
Epoch 4/100
363/363 [==============================] - 0s 1ms/step - loss: 0.2996 - val_loss: 0.3040
Epoch 5/100
363/363 [==============================] - 0s 1ms/step - loss: 0.2976 - val_loss: 0.2990
Epoch 6/100
363/363 [==============================] - 0s 1ms/step - loss: 0.2966 - val_loss: 0.3024
Epoch 7/100
363/363 [==============================] - 0s 1ms/step - loss: 0.2962 - val_loss: 0.3180
Epoch 8/100
363/363 [==============================] - 0s 1ms/step - loss: 0.2957 - val_loss: 0.2993
Epoch 9/100
363/363 [==============================] - 0s 1ms/step - loss: 0.2971 - val_loss: 0.3050
Epoch 10/100
363/363 [==============================] - 0s 1ms/step - loss: 0.2940 - val_lo

In [115]:
!del california_housing_model.h5

In [143]:
def generate_path():
    root_logdir = os.path.join('..\\', 'TensorBoard_logs', time.strftime('Logs_%Y_%m_%d-%H_%M_%S'))
    return root_logdir

tensorboard_cb = keras.callbacks.TensorBoard(generate_path())

In [144]:
history = model.fit(X_train, y_train,
                   epochs=10,
                   validation_data=(X_valid, y_valid),
                   callbacks=[tensorboard_cb])

Epoch 1/10
363/363 [==============================] - 1s 1ms/step - loss: 0.2823 - val_loss: 0.2959
Epoch 2/10
363/363 [==============================] - 0s 1ms/step - loss: 0.2823 - val_loss: 0.2896
Epoch 3/10
363/363 [==============================] - 0s 1ms/step - loss: 0.2829 - val_loss: 0.2906
Epoch 4/10
363/363 [==============================] - 0s 1ms/step - loss: 0.2825 - val_loss: 0.2915
Epoch 5/10
363/363 [==============================] - 0s 1ms/step - loss: 0.2828 - val_loss: 0.2910
Epoch 6/10
363/363 [==============================] - 0s 1ms/step - loss: 0.2811 - val_loss: 0.2954
Epoch 7/10
363/363 [==============================] - 0s 1ms/step - loss: 0.2820 - val_loss: 0.2939
Epoch 8/10
363/363 [==============================] - 0s 1ms/step - loss: 0.2820 - val_loss: 0.3066
Epoch 9/10
363/363 [==============================] - 0s 1ms/step - loss: 0.2815 - val_loss: 0.2911
Epoch 10/10
363/363 [==============================] - 0s 1ms/step - loss: 0.2793 - val_loss: 0.2932

In [149]:
!tensorboard --logdir=C:/Users/Lenovo/PycharmProjects/DeepLearning/TensorBoard_logs --port=6006

^C


In [152]:
%load_ext tensorboard
%tensorboard --logdir=C:/Users/Lenovo/PycharmProjects/DeepLearning/TensorBoard_logs --port=6006

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 3908), started 0:01:08 ago. (Use '!kill 3908' to kill it.)